In [1]:
import pyspark
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql import SQLContext
from pyspark import SparkConf
from pyspark.sql.functions import regexp_replace
import pymongo
import pandas as pd

In [2]:
# In phiên bản của PySpark
print("PySpark version:", pyspark.__version__)

PySpark version: 3.5.3


In [3]:
# Tạo đối tượng cấu hình SparkConf
conf= pyspark.SparkConf().set("spark.jars.packages", "org.mongodb.spark:mongo-spark-connector_2.12:3.0.1,org.postgresql:postgresql:42.7.4")\
.setMaster("local[*]").setAppName("ReadMongoDB")\
.setAll([("spark.driver.memory","1g"),("spark.executor.memory","1g")])

# Dừng SparkContext hiện tại nếu có
if SparkContext._active_spark_context:
    SparkContext._active_spark_context.stop()

# Tạo SparkContext mới với cấu hình
sc = SparkContext(conf=conf)

In [4]:
sqlC = SQLContext(sc)

c:\Users\tram2\AppData\Local\Programs\Python\Python310\lib\site-packages\pyspark\sql\context.py:113: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [5]:
mongo_ip = "mongodb://localhost:27017/rugs."

In [6]:
rug = sqlC.read.format("com.mongodb.spark.sql.DefaultSource").option("uri", mongo_ip +"rugs_collection").load()

In [7]:
rug.show()

+----------------+--------------------+--------------------+---------+-------+--------------------+-----------+--------------------+-------+-------------+--------------------+------+--------------------+
|Cleaning_Process|            Material|                Name|Old_Price| Origin|             Pattern|Pile_height|     Product_Reviews|   Save|Special_Price|                Type|Weight|                 _id|
+----------------+--------------------+--------------------+---------+-------+--------------------+-----------+--------------------+-------+-------------+--------------------+------+--------------------+
|   Easy to clean|           Polyester|Cleo 013-0016 619...|    223.2|Belgium|            Abstract|       25.0|                 N/A|Save 24|       169.08|              Modern|  2900|{6708ecdbc59a34b2...|
|   Easy to clean|       Polypropylene|Nova NV10 Antique...|     59.0| Turkey|              Floral|        9.0|Excellent distres...|Save 33|         39.6|         Traditional|  2250|{6

In [8]:
rug.printSchema()

root
 |-- Cleaning_Process: string (nullable = true)
 |-- Material: string (nullable = true)
 |-- Name: string (nullable = true)
 |-- Old_Price: double (nullable = true)
 |-- Origin: string (nullable = true)
 |-- Pattern: string (nullable = true)
 |-- Pile_height: double (nullable = true)
 |-- Product_Reviews: string (nullable = true)
 |-- Save: string (nullable = true)
 |-- Special_Price: double (nullable = true)
 |-- Type: string (nullable = true)
 |-- Weight: integer (nullable = true)
 |-- _id: struct (nullable = true)
 |    |-- oid: string (nullable = true)



In [9]:
#Xử lý dữ liệu rỗng 
rug_cleand=rug.filter(
    F.col('Name').isNotNull() & 
    F.col('Old_Price').isNotNull() & 
    F.col('Special_Price').isNotNull() & 
    F.col('Save').isNotNull() & 
    F.col('Material').isNotNull() & 
    F.col('Cleaning_Process').isNotNull() & 
    F.col('Pattern').isNotNull() & 
    F.col('Pile_height').isNotNull() & 
    F.col('Weight').isNotNull() &
    F.col('Origin').isNotNull() &
    F.col('Type').isNotNull() &
    F.col('Product_Reviews').isNotNull()
)


In [10]:
rug_cleand.show()

+--------------------+--------------------+--------------------+---------+-------+--------------------+-----------+--------------------+-------+-------------+--------------------+------+--------------------+
|    Cleaning_Process|            Material|                Name|Old_Price| Origin|             Pattern|Pile_height|     Product_Reviews|   Save|Special_Price|                Type|Weight|                 _id|
+--------------------+--------------------+--------------------+---------+-------+--------------------+-----------+--------------------+-------+-------------+--------------------+------+--------------------+
|       Easy to clean|           Polyester|Cleo 013-0016 619...|    223.2|Belgium|            Abstract|       25.0|                 N/A|Save 24|       169.08|              Modern|  2900|{6708ecdbc59a34b2...|
|       Easy to clean|       Polypropylene|Nova NV10 Antique...|     59.0| Turkey|              Floral|        9.0|Excellent distres...|Save 33|         39.6|         T

In [11]:
rug_cleand.show()

+--------------------+--------------------+--------------------+---------+-------+--------------------+-----------+--------------------+-------+-------------+--------------------+------+--------------------+
|    Cleaning_Process|            Material|                Name|Old_Price| Origin|             Pattern|Pile_height|     Product_Reviews|   Save|Special_Price|                Type|Weight|                 _id|
+--------------------+--------------------+--------------------+---------+-------+--------------------+-----------+--------------------+-------+-------------+--------------------+------+--------------------+
|       Easy to clean|           Polyester|Cleo 013-0016 619...|    223.2|Belgium|            Abstract|       25.0|                 N/A|Save 24|       169.08|              Modern|  2900|{6708ecdbc59a34b2...|
|       Easy to clean|       Polypropylene|Nova NV10 Antique...|     59.0| Turkey|              Floral|        9.0|Excellent distres...|Save 33|         39.6|         T

In [12]:
rug_cleand = rug.na.replace("N/A", "Unknown").filter(
    F.col('Name').isNotNull() & 
    F.col('Old_Price').isNotNull() & 
    F.col('Special_Price').isNotNull() & 
    F.col('Save').isNotNull() & 
    F.col('Material').isNotNull() & 
    F.col('Cleaning_Process').isNotNull() & 
    F.col('Pattern').isNotNull() & 
    F.col('Pile_height').isNotNull() & 
    F.col('Weight').isNotNull() &
    F.col('Origin').isNotNull() &
    F.col('Type').isNotNull() &
    F.col('Product_Reviews').isNotNull()
)


In [13]:
rug_cleand.show()

+--------------------+--------------------+--------------------+---------+-------+--------------------+-----------+--------------------+-------+-------------+--------------------+------+--------------------+
|    Cleaning_Process|            Material|                Name|Old_Price| Origin|             Pattern|Pile_height|     Product_Reviews|   Save|Special_Price|                Type|Weight|                 _id|
+--------------------+--------------------+--------------------+---------+-------+--------------------+-----------+--------------------+-------+-------------+--------------------+------+--------------------+
|       Easy to clean|           Polyester|Cleo 013-0016 619...|    223.2|Belgium|            Abstract|       25.0|             Unknown|Save 24|       169.08|              Modern|  2900|{6708ecdbc59a34b2...|
|       Easy to clean|       Polypropylene|Nova NV10 Antique...|     59.0| Turkey|              Floral|        9.0|Excellent distres...|Save 33|         39.6|         T

In [14]:
rug_cleand = rug_cleand.withColumn('Save', regexp_replace('Save', 'Save ', ''))



In [15]:
rug_cleand.show()

+--------------------+--------------------+--------------------+---------+-------+--------------------+-----------+--------------------+----+-------------+--------------------+------+--------------------+
|    Cleaning_Process|            Material|                Name|Old_Price| Origin|             Pattern|Pile_height|     Product_Reviews|Save|Special_Price|                Type|Weight|                 _id|
+--------------------+--------------------+--------------------+---------+-------+--------------------+-----------+--------------------+----+-------------+--------------------+------+--------------------+
|       Easy to clean|           Polyester|Cleo 013-0016 619...|    223.2|Belgium|            Abstract|       25.0|             Unknown|  24|       169.08|              Modern|  2900|{6708ecdbc59a34b2...|
|       Easy to clean|       Polypropylene|Nova NV10 Antique...|     59.0| Turkey|              Floral|        9.0|Excellent distres...|  33|         39.6|         Traditional|  22

In [16]:
rug_cleand = rug.withColumn('Old_Price', F.col('Old_Price').cast('float')) \
                .withColumn('Pile_height', F.col('Pile_height').cast('float')) \
                .withColumn('Special_Price', F.col('Special_Price').cast('float'))


In [17]:
rug_cleand = rug_cleand.drop("_id")

In [18]:
rug_cleand.show()

+----------------+--------------------+--------------------+---------+-------+--------------------+-----------+--------------------+-------+-------------+--------------------+------+
|Cleaning_Process|            Material|                Name|Old_Price| Origin|             Pattern|Pile_height|     Product_Reviews|   Save|Special_Price|                Type|Weight|
+----------------+--------------------+--------------------+---------+-------+--------------------+-----------+--------------------+-------+-------------+--------------------+------+
|   Easy to clean|           Polyester|Cleo 013-0016 619...|    223.2|Belgium|            Abstract|       25.0|                 N/A|Save 24|       169.08|              Modern|  2900|
|   Easy to clean|       Polypropylene|Nova NV10 Antique...|     59.0| Turkey|              Floral|        9.0|Excellent distres...|Save 33|         39.6|         Traditional|  2250|
|   Easy to clean|Polyester , Polyp...|Aurora AU17 Linea...|     89.0| Turkey|       

In [19]:
# Tạo DataFrame 'Rugs' chứa thông tin chi tiết về sản phẩm
Rugs = rug_cleand.select(
    F.monotonically_increasing_id().alias("rug_id"),
    "Name",
    "Old_Price",
    "Special_Price",
    "Save",
    "Material",
    "Cleaning_Process",
    "Pile_height",
    "Weight",
    "Origin"
    
).distinct().orderBy("rug_id")

# Tạo DataFrame 'Review' chứa đánh giá sản phẩm
Review = rug_cleand.select(
    F.monotonically_increasing_id().alias("rug_id"),
    "Name",
    "Product_Reviews"
).distinct().orderBy("rug_id")

# Tạo DataFrame 'Type'
Type = rug_cleand.select(
    F.monotonically_increasing_id().alias("rug_id"),
    "Type"
).distinct().orderBy("rug_id")

# Tạo DataFrame 'Material'
Material = rug_cleand.select(
    F.monotonically_increasing_id().alias("rug_id"),
    "Material"
).distinct().orderBy("rug_id")

# Tạo DataFrame 'Pattern'
Pattern = rug_cleand.select(
    F.monotonically_increasing_id().alias("rug_id"),
    "Pattern"
).distinct().orderBy("rug_id")

In [20]:
# Kết nối tới PostgreSQL
jdbc_url = "jdbc:postgresql://localhost:5432/rugs"  
connection_properties = {
    "user": "tram2",        
    "password": "1111",      
    "driver": "org.postgresql.Driver"
}


In [21]:
# 5. Ghi dữ liệu vào bảng 'rug_details' (Thông tin về thảm)
Rugs.write.jdbc(
    url=jdbc_url,
    table="rug_details", 
    mode="overwrite",  # Ghi đè bảng nếu đã tồn tại, có thể thay bằng 'append' nếu muốn chèn thêm
    properties=connection_properties
)

In [22]:
# 6. Ghi dữ liệu vào bảng 'rug_reviews' (Đánh giá sản phẩm)
Review.write.jdbc(
    url=jdbc_url,
    table="rug_reviews", 
    mode="overwrite",  # Ghi đè bảng nếu đã tồn tại
    properties=connection_properties
)

In [23]:
# Ghi dữ liệu vào bảng 'type_details'
Type.write.jdbc(
    url=jdbc_url,
    table="type_details", 
    mode="overwrite",  # Ghi đè bảng nếu đã tồn tại, có thể thay bằng 'append' nếu muốn chèn thêm
    properties=connection_properties
)

# Ghi dữ liệu vào bảng 'material_details'
Material.write.jdbc(
    url=jdbc_url,
    table="material_details", 
    mode="overwrite",  # Ghi đè bảng nếu đã tồn tại, có thể thay bằng 'append' nếu muốn chèn thêm
    properties=connection_properties
)

# Ghi dữ liệu vào bảng 'pattern_details'
Pattern.write.jdbc(
    url=jdbc_url,
    table="pattern_details", 
    mode="overwrite",  # Ghi đè bảng nếu đã tồn tại, có thể thay bằng 'append' nếu muốn chèn thêm
    properties=connection_properties
)


In [24]:
def write_to_postgres(df, table_name):
    try:
        df.write.jdbc(
            url=jdbc_url,
            table=table_name,
            mode="overwrite",
            properties=connection_properties
        )
        print(f"Đã ghi thành công bảng {table_name} vào PostgreSQL!")
    except Exception as e:
        print(f"Lỗi khi ghi bảng {table_name} vào PostgreSQL: {e}")

write_to_postgres(Rugs, "rug_details")
write_to_postgres(Review, "rug_reviews")
write_to_postgres(Material, "material_details")
write_to_postgres(Type, "type_details")
write_to_postgres(Pattern, "pattern_details")


Đã ghi thành công bảng Rugs vào PostgreSQL!
Đã ghi thành công bảng Review vào PostgreSQL!
Đã ghi thành công bảng Material vào PostgreSQL!
Đã ghi thành công bảng Type vào PostgreSQL!
Đã ghi thành công bảng Pattern vào PostgreSQL!
